In [1]:
# [+] SparkSession 설정
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('udf').getOrCreate()

In [19]:
# 샘플 데이터: 한신대 맛집 및 대표메뉴
data_redvelvet = [
    ('배주현',32),
    ('강슬기',29),
    ('손승완',29),
    ('박수영',26),
    ('김예림',24),
    ('새멤버',50),
    ('새멤버2',60)
]

In [20]:
# [+] 스키마 정의
schema = ['member_name', 'age']

In [21]:
# [+] 데이터프레임 생성
df = spark.createDataFrame(data=data_redvelvet, schema=schema)

In [22]:
# [+] 데이터프레임 출력
df.show()

+-----------+---+
|member_name|age|
+-----------+---+
|     배주현| 32|
|     강슬기| 29|
|     손승완| 29|
|     박수영| 26|
|     김예림| 24|
|     새멤버| 50|
|    새멤버2| 60|
+-----------+---+



In [23]:
# [+] 데이터프레임 스키마 출력
df.printSchema()

root
 |-- member_name: string (nullable = true)
 |-- age: long (nullable = true)



In [24]:
# [+] Temporary View 생성
df.createOrReplaceTempView('age_redvelvet')

### User Defined Function 생성
1. translate(): Google Translation API를 이용하여 한글 식당 이름을 영문으로 번역하는 함수
2. scale_score(): 5점 만점 평점을 100점 스케일로 변환하는 함수

In [25]:
# Google translation 라이브러리 설치
!pip install googletrans==4.0.0-rc1

In [26]:
# [+] Google translator 임포트
from googletrans import Translator

translator = Translator()

In [27]:
# [+] 번역 테스트
result = translator.translate('찌개 동아리', src='ko', dest='en')
print(result)

Translated(src=ko, dest=en, text=Stew, pronunciation=None, extra_data="{'confiden...")


In [28]:
# [+] 번역 결과 출력
result.text

'Stew'

In [29]:
# [+] extra_data 출력
result.extra_data

{'confidence': None,
 'parts': [<googletrans.models.TranslatedPart at 0x22f98ef7670>],
 'origin_pronunciation': 'jjigae dong-ali',
 'parsed': [['jjigae dong-ali',
   None,
   None,
   [[[0, [[[None, 6]], [True]]]], 6],
   [['찌개 동아리', None, None, 6]],
   None,
   ['찌개 동아리', 'ko', 'en', True]],
  [[[None,
     None,
     None,
     None,
     None,
     [['Stew',
       None,
       None,
       None,
       [['Stew', [5], []], ['Stew club', [11], []]]]]]],
   'en',
   1,
   'ko',
   ['찌개 동아리', 'ko', 'en', True]],
  'ko']}

In [30]:
# [+] 영어 발음(pronunciation) 출력
result.extra_data.get('origin_pronunciation')

'jjigae dong-ali'

In [31]:
# [+] UDF 1: 나이 구분

def age_category(age):
    if age < 35:
        return 'young'
    elif age > 35 and age <59 :
        return 'adult'
    else :
        return 'senior'

In [32]:
# [+] UDF 등록
spark.udf.register('age_category', age_category)

<function __main__.age_category(age)>

In [33]:
# [+] SQL문 처리
spark.sql("SELECT member_name, age_category(age) AS member_name \
        FROM age_redvelvet").show()

+-----------+-----------+
|member_name|member_name|
+-----------+-----------+
|     배주현|      young|
|     강슬기|      young|
|     손승완|      young|
|     박수영|      young|
|     김예림|      young|
|     새멤버|      adult|
|    새멤버2|     senior|
+-----------+-----------+

